In [6]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import pandas as pd
import sys

from dataframe import preprocess, rename_header_dataframe

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

from database import connect, create_table
from models import DataSP

## Database connection

In [8]:
engine, session = connect()
engine.echo = False

2023-03-24 16:19:45,036 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-24 16:19:45,036 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:19:45,038 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-24 16:19:45,039 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:19:45,040 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-24 16:19:45,041 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load CSV files

In [9]:
filename = '../csv/original.csv'
df = pd.read_csv(filename, sep=';', low_memory=False, skipinitialspace=True)

## Create table DataSP and insert rows of sheet in db

In [10]:
def create_datasp(info):
    return DataSP(seq=info['seq'],
                  modified=info['modified'], institution_code=info['institution_code'],
                  collection_code=info['collection_code'], catalog_number=info['catalog_number'],
                  basis_of_record=info['basis_of_record'], kingdom=info['kingdom'], phylum=info['phylum'],
                  classe=info['class'], order=info['order'], family=info['family'],
                  genus=info['genus'],
                  specific_epithet=info['specific_epithet'],
                  infraspecific_epithet=info['infraspecific_epithet'],
                  scientific_name=info['scientific_name'],
                  scientific_name_authorship=info['scientific_name_authorship'],
                  identified_by=info['identified_by'], year_identified=info['year_identified'],
                  month_identified=info['month_identified'], day_identified=info['day_identified'],
                  type_status=info['type_status'],
                  recorded_by=info['recorded_by'], record_number=info['record_number'],
                  field_number=info['field_number'], year=info['year'], month=info['month'],
                  day=info['day'], event_time=info['event_time'],
                  continent_ocean=info['continent_ocean'], country=info['country'],
                  state_province=info['state_province'], county=info['county'], locality=info['locality'],
                  decimal_longitude=info['decimal_longitude'],
                  decimal_latitude=info['decimal_latitude'], verbatim_longitude=info['verbatim_longitude'],
                  verbatim_latitude=info['verbatim_latitude'],
                  coordinate_precision=info['coordinate_precision'],
                  bounding_box=info['bounding_box'],
                  minimum_elevation_in_meters=info['minimum_elevation_in_meters'],
                  maximum_elevation_in_meters=info['maximum_elevation_in_meters'],
                  minimum_depth_in_meters=info['minimum_depth_in_meters'],
                  maximum_depth_in_meters=info['maximum_depth_in_meters'], sex=info['sex'],
                  preparation_type=info['preparation_type'],
                  individual_count=info['individual_count'],
                  previous_catalog_number=info['previous_catalog_number'],
                  relationship_type=info['relationship_type'],
                  related_catalog_item=info['related_catalog_item'],
                  occurrence_remarks=info['occurrence_remarks'], barcode=info['barcode'],
                  imagecode=info['imagecode'], geo_flag=info['geo_flag'])

create_table(engine, DataSP)

count_records_in_data_sp = session.query(DataSP).count()

if count_records_in_data_sp == 0:
    rename_header_dataframe(df)
    df = preprocess(df)
    for row in df.iterrows():
        try:
            data_sp = create_datasp(row[1])
            session.add(data_sp)
            session.commit()
        except:
            session.flush()
    count_records_in_data_sp = session.query(DataSP).count()
    print('count records inserted in table %s: %d' % (DataSP.__tablename__, count_records_in_data_sp))

create table: data_sp
count records inserted in table data_sp: 55453


## Close connection

In [11]:
session.close()
engine.dispose()